# 中文 CLIP

https://github.com/OFA-Sys/Chinese-CLIP

直接下载该仓库处理好的数据集

## 预测及评估 验证集 valid

### 图文特征提取

In [6]:
!export CUDA_VISIBLE_DEVICES=0
!export PYTHONPATH=${PYTHONPATH}:`pwd`/cn_clip

!python -u cn_clip/eval/extract_features.py \
    --extract-image-feats \
    --extract-text-feats \
    --image-data="/data/nxy/datasets/MUGE/lmdb/valid/imgs" \
    --text-data="/data/nxy/datasets/MUGE/valid_texts.jsonl" \
    --img-batch-size=32 \
    --text-batch-size=32 \
    --context-length=52 \
    --resume="/data/nxy/pretrained_weights/clip_cn_vit-b-16.pt" \
    --vision-model=ViT-B-16 \
    --text-model=RoBERTa-wwm-ext-base-chinese

Params:
  context_length: 52
  debug: False
  extract_image_feats: True
  extract_text_feats: True
  image_data: /data/nxy/datasets/MUGE/lmdb/valid/imgs
  image_feat_output_path: None
  img_batch_size: 32
  precision: amp
  resume: /data/nxy/pretrained_weights/clip_cn_vit-b-16.pt
  text_batch_size: 32
  text_data: /data/nxy/datasets/MUGE/valid_texts.jsonl
  text_feat_output_path: None
  text_model: RoBERTa-wwm-ext-base-chinese
  vision_model: ViT-B-16
Loading vision model config from cn_clip/clip/model_configs/ViT-B-16.json
Loading text model config from cn_clip/clip/model_configs/RoBERTa-wwm-ext-base-chinese.json
Preparing image inference dataset.
Preparing text inference dataset.
Begin to load model checkpoint from /data/nxy/pretrained_weights/clip_cn_vit-b-16.pt.
=> loaded checkpoint '/data/nxy/pretrained_weights/clip_cn_vit-b-16.pt' (epoch 15 @ 180000 steps)
Make inference for texts...
100%|█████████████████████████████████████████| 157/157 [00:07<00:00, 21.94it/s]
5008 text featur

### KNN检索

In [7]:
!python -u cn_clip/eval/make_topk_predictions.py \
    --image-feats="/data/nxy/datasets/MUGE/valid_imgs.img_feat.jsonl" \
    --text-feats="/data/nxy/datasets/MUGE/valid_texts.txt_feat.jsonl" \
    --top-k=10 \
    --eval-batch-size=32768 \
    --output="/data/nxy/datasets/MUGE/valid_predictions.jsonl"

Params:
  eval_batch_size: 32768
  image_feats: /data/nxy/datasets/MUGE/valid_imgs.img_feat.jsonl
  output: /data/nxy/datasets/MUGE/valid_predictions.jsonl
  text_feats: /data/nxy/datasets/MUGE/valid_texts.txt_feat.jsonl
  top_k: 10
Begin to load image features...
29806it [00:04, 6941.37it/s]
Finished loading image features.
Begin to compute top-10 predictions for texts...
5008it [02:13, 37.55it/s]
Top-10 predictions are saved in /data/nxy/datasets/MUGE/valid_predictions.jsonl
Done!


### Recall计算

In [8]:
!python cn_clip/eval/evaluation.py \
        /data/nxy/datasets/MUGE/valid_texts.jsonl \
        /data/nxy/datasets/MUGE/valid_predictions.jsonl \
        output.json

Read standard from /data/nxy/datasets/MUGE/valid_texts.jsonl
Read user submit file from /data/nxy/datasets/MUGE/valid_predictions.jsonl
The evaluation finished successfully.


## 预测及评估 测试集 test

### 数据处理

In [3]:
!python cn_clip/preprocess/build_lmdb_dataset.py \
    --data_dir /data/nxy/datasets/MUGE \
    --splits test

Dataset splits to be processed: test
30399it [00:46, 648.85it/s] 
Finished serializing 30399 test split images into /data/nxy/datasets/MUGE/lmdb/test/imgs.
done!


### 模型训练

In [4]:
!bash run_scripts/muge_finetune_vit-b-16_rbt-base.sh /data/nxy/

/home/nxy/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/torch/distributed/launch.py:188: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Loading vision model config from cn_clip/clip/model_configs/ViT-B-16.json
Loading text model config from cn_clip/clip/model_configs/RoBERTa-wwm-ext-base-chinese.json
2022-11-22,13:17:09 | INFO | Rank 0 | train LMDB file contains 129380 images and 250314 pairs.
2022-11-22,13:17:09 | INFO | Rank 0 | val LMDB file contains 29806 images and 30588 pairs.
2022-11-22,13:17:09 | INFO | Rank 0 | Params:
2022-11-22,13:17:09 | INFO | Rank 0 |   aggregate: True
2022-11-22,13:17:09 | INFO | Rank 0 |   batch_size:

### 图文特征提取

In [5]:
!python -u cn_clip/eval/extract_features.py \
    --extract-image-feats \
    --extract-text-feats \
    --image-data="/data/nxy/datasets/MUGE/lmdb/test/imgs" \
    --text-data="/data/nxy/datasets/MUGE/test_texts.jsonl" \
    --img-batch-size=32 \
    --text-batch-size=32 \
    --context-length=52 \
    --resume="/data/nxy/experiments/muge_finetune_vit-b-16_roberta-base_bs128_1gpu/checkpoints/epoch_latest.pt" \
    --vision-model=ViT-B-16 \
    --text-model=RoBERTa-wwm-ext-base-chinese

Params:
  context_length: 52
  debug: False
  extract_image_feats: True
  extract_text_feats: True
  image_data: /data/nxy/datasets/MUGE/lmdb/test/imgs
  image_feat_output_path: None
  img_batch_size: 32
  precision: amp
  resume: /data/nxy/experiments/muge_finetune_vit-b-16_roberta-base_bs128_1gpu/checkpoints/epoch_latest.pt
  text_batch_size: 32
  text_data: /data/nxy/datasets/MUGE/test_texts.jsonl
  text_feat_output_path: None
  text_model: RoBERTa-wwm-ext-base-chinese
  vision_model: ViT-B-16
Loading vision model config from cn_clip/clip/model_configs/ViT-B-16.json
Loading text model config from cn_clip/clip/model_configs/RoBERTa-wwm-ext-base-chinese.json
Preparing image inference dataset.
Preparing text inference dataset.
Begin to load model checkpoint from /data/nxy/experiments/muge_finetune_vit-b-16_roberta-base_bs128_1gpu/checkpoints/epoch_latest.pt.
=> loaded checkpoint '/data/nxy/experiments/muge_finetune_vit-b-16_roberta-base_bs128_1gpu/checkpoints/epoch_latest.pt' (epoch 3 

### KNN检索

In [7]:
!python -u cn_clip/eval/make_topk_predictions.py \
    --image-feats="/data/nxy/datasets/MUGE/test_imgs.img_feat.jsonl" \
    --text-feats="/data/nxy/datasets/MUGE/test_texts.txt_feat.jsonl" \
    --top-k=10 \
    --eval-batch-size=32768 \
    --output="/data/nxy/datasets/MUGE/test_predictions.jsonl"

Params:
  eval_batch_size: 32768
  image_feats: /data/nxy/datasets/MUGE/test_imgs.img_feat.jsonl
  output: /data/nxy/datasets/MUGE/test_predictions.jsonl
  text_feats: /data/nxy/datasets/MUGE/test_texts.txt_feat.jsonl
  top_k: 10
Begin to load image features...
30399it [00:32, 928.96it/s] 
Finished loading image features.
Begin to compute top-10 predictions for texts...
5004it [02:14, 37.14it/s]
Top-10 predictions are saved in /data/nxy/datasets/MUGE/test_predictions.jsonl
Done!
